# `LibreASR`

## Imports

In [ ]:
%reload_ext autoreload
%autoreload 3
# %matplotlib inline

In [ ]:
from lib.imports import *

## `Config`

In [ ]:
conf, lang, builder_train, builder_valid, db, m, learn = parse_and_apply_config()
(conf, lang, db, m, learn)

In [ ]:
builder_train.print()
builder_valid.print()

In [ ]:
builder_train.plot()

In [ ]:
tpl = db.one_batch()
X, Ym, _, _ = tpl[0]
Y, Y_lens, X_lens = tpl[1]
what(X), what(X_lens), what(Y), what(Y_lens)

In [ ]:
X_lens, Y_lens

### `Augmentation`

In [ ]:
# db.aug(n=3) #0: signal, 7: spectro

## `Model`

In [ ]:
# switch to gpu mode
# m.convert_to_gpu()

In [ ]:
m = m.to(conf["cuda"]["device"])

In [ ]:
print(m)
print("n_params:", n_params(m))

Timing

In [ ]:
%%time
_in = db.one_batch()[0]

In [ ]:
%%time
with torch.no_grad():
    _out = m(cudaize(_in))
    print("X", _in[0].shape, "->", _out.shape)
    print("Y", _in[1].shape)

## `Train`

In [ ]:
ckpt = "model"

In [ ]:
try:
    learn.load(ckpt, with_opt=True, strict=False)
    pass
except Exception as e:
    print("failed loading ckpt", ckpt)
    print(e)

### `fit`

In [ ]:
learn.unfreeze()

In [ ]:
# learn.lr_find()

In [ ]:
# hypers
do_warmup = True
epochs = 20
warmup_epochs = 2.9
warmup = int(warmup_epochs + 1)

lr = 7e-4
wd = 0.01 # 0.1

div = 500. # 100.
div_final = 1.001

pct_start = warmup_epochs / warmup

In [ ]:
# warmup
if do_warmup:
    try:
        learn.fit_one_cycle(warmup, lr, wd=wd, cbs=learn.extra_cbs, div_final=div_final, div=div, pct_start=pct_start)
    except: pass
    m = m.to(conf["cuda"]["device"])
    learn.save(ckpt, with_opt=True)

In [ ]:
# tune
m = m.to(conf["cuda"]["device"])
learn.load(ckpt, with_opt=True, strict=False, device=conf["cuda"]["device"])
learn.fit(epochs, lr, wd=wd, cbs=learn.extra_cbs)

In [ ]:
raise Exception("done")

### `checks`

In [ ]:
learn.recorder.values

In [ ]:
# save model for later
# learn.save(checkpoints[-1], with_opt=True)
learn.save(ckpt, with_opt=False)

## `Decode`

In [ ]:
# final
_ = list(learn.test(min_samples=400, train=False))

In [ ]:
ylens = map(lambda x: x.get("text/ground_truth"), _)
ylens = filter(lambda x: x is not None and len(x) != 0, ylens)
ylens = map(lambda x: len(x), ylens)
ylens = np.array(list(ylens))

wers = map(lambda x: x.get("metrics/wer", None), _)
wers = filter(lambda x: x is not None, wers)
wers = np.array(list(wers))
ylens[:3], wers[:3], len(ylens), len(wers)

In [ ]:
stack = np.stack([ylens, wers]).T
stack.shape

In [ ]:
df = pd.DataFrame(stack, columns=["ylen", "wer"])
df.head()

In [ ]:
grouped = df.groupby("ylen").mean()["wer"].values

In [ ]:
plt.plot(grouped)

In [ ]:
# _[:10]

In [ ]:
# model
_ = list(learn.test(min_samples=256*4, train=False))

In [ ]:
m.joint.joint[2].bias

In [ ]:
m.joint.joint[2].bias

## Archive

In [ ]:
from lib.model_utils import *
from pathlib import Path

In [ ]:
save_asr_model()